# Hotstart

Many times one needs to split a simulation in intermittend steps. This is true for long hindacts and forecasts. This Notebook outlines the hotstart process using the schism solver. 


- Requires:

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.model as pm

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env PATH. This is not needed when we are running from python.

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
import numpy
cpath = numpy.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
import geopandas as gp
# use cartopy to get coastlines
import cartopy.feature as cf

Without loss of generality we are using the natural earth's intermediate resolution dataset as coastlines

In [ ]:
cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

## Setup

The minimum information required is an initial model and corresponding run. We set this one up. For more info see the [MODEL](./MODEL.ipynb) Notebook.


In [ ]:
#define in a dictionary the properties of the model..
model={'solver':'schism',
     'geometry':{'lon_min' : -25,'lon_max' : -12.,'lat_min' : 56.,'lat_max' : 74.}, 
#     'manning':.12, #uniform manning value
#     'windrot':0.00001, #uniform windrot value
     'coastlines':ne_i,
     'mesh_generator' : 'jigsaw', # set grid generator 
     'meteo_engine':'cfgrib',
     'start_date':'2018-10-1 0:0:0',
     'time_frame':'24H',
     'meteo_source' : './data/uvp_2018100100.grib', #path to meteo files
     'dem_source' : './data/dem.nc', #path to dem file
     'rpath':'./test/hotstart/step1/', #location of calc folder
#     'update':['all'], #set which component should be updated  (meteo,dem,model)
#     'monitor':True, # get time series for observation points
     'parameters':{'dt':400, 'rnday':1., 'nhot':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'nhot_write':108 } # set param.in components
    }

* Note that the parameters arguments will override relevant settings

## Initialize

In [ ]:
#initialize a model
b = pm.set(**model)

In [ ]:
#Execute
b.execute()

*Note* that hotstart files have been created (check ./test/hotstart/step1/outputs)

## Use the cast module to do a hotstart

In [ ]:
from pyposeidon.utils import cast
import pandas as pd
import datetime

In [ ]:
# folder for the new run hotstarting from the step1 folder
new_path = './test/hotstart/step2/'

In [ ]:
# new meteo file
mf = './data/uvp_2018100112.grib'

In [ ]:
rs = cast.set(solver='schism',
        model=b, #reference model
        ppath=b.rpath, #original path
        cpath=new_path, #new path
        meteo=mf, #new meteo
        sdate=b.date + pd.DateOffset(hours=12) #new start time
            ) 



In [ ]:
rs.run(execute=True)